In [13]:
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
import numpy
import keras
import matplotlib.pyplot
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(332, 332, 3))
# Create the model
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)
model = models.Sequential()
# Add the vgg convolutional base model
model.add(vgg_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

<keras.engine.input_layer.InputLayer object at 0x7ff770078320> False
<keras.layers.convolutional.Conv2D object at 0x7ff770078a20> False
<keras.layers.convolutional.Conv2D object at 0x7ff7700786d8> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff770078d68> False
<keras.layers.convolutional.Conv2D object at 0x7ff770078b70> False
<keras.layers.convolutional.Conv2D object at 0x7ff7d8c1c668> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff7d8a85780> False
<keras.layers.convolutional.Conv2D object at 0x7ff7d8a85278> False
<keras.layers.convolutional.Conv2D object at 0x7ff7144257b8> False
<keras.layers.convolutional.Conv2D object at 0x7ff714528438> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff714528cf8> False
<keras.layers.convolutional.Conv2D object at 0x7ff714528b38> False
<keras.layers.convolutional.Conv2D object at 0x7ff71451d6a0> False
<keras.layers.convolutional.Conv2D object at 0x7ff71451de48> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff714510978>

In [14]:
import os
base_dir = '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir2=os.path.join(train_dir, 'airplane/1382740959_a8eb7edb6c.jpg')

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen  = ImageDataGenerator(rescale=1./255)
img = keras.preprocessing.image.load_img(train_dir2)
img_array = keras.preprocessing.image.img_to_array(img)

print(img_array.shape, img_array.dtype)


# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    #color_mode='grayscale',
                                                    target_size=(332,332),
                                                    shuffle=True)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=20,
                                                        class_mode='categorical',
                                                       # color_mode='grayscale',
                                                        target_size=(332,332)) 

(333, 499, 3) float32
Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [16]:
import os
base_dir = '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [17]:
# Compile the model
import keras
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

checkpoint = keras.callbacks.callbacks.ModelCheckpoint('model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
callbacks_list = [checkpoint]
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=4,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
      callbacks=callbacks_list)
 


Epoch 1/4
125/124 [==============================] - 24s 189ms/step - loss: 1.0263 - acc: 0.6199 - val_loss: 0.2137 - val_acc: 0.7653

Epoch 00001: val_acc improved from -inf to 0.76527, saving model to model.hdf5
Epoch 2/4
125/124 [==============================] - 23s 185ms/step - loss: 0.5491 - acc: 0.8264 - val_loss: 0.5528 - val_acc: 0.8296

Epoch 00002: val_acc improved from 0.76527 to 0.82958, saving model to model.hdf5
Epoch 3/4
125/124 [==============================] - 23s 184ms/step - loss: 0.4108 - acc: 0.8709 - val_loss: 0.0476 - val_acc: 0.8617

Epoch 00003: val_acc improved from 0.82958 to 0.86174, saving model to model.hdf5
Epoch 4/4
125/124 [==============================] - 23s 184ms/step - loss: 0.2823 - acc: 0.9166 - val_loss: 0.5035 - val_acc: 0.8875

Epoch 00004: val_acc improved from 0.86174 to 0.88746, saving model to model.hdf5


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from keras.preprocessing import image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv

model = tf.keras.models.load_model('/kaggle/working/model.hdf5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk('/kaggle/input/duth-cv-2019-2020-hw-4/vehicles/test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(332,332), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        #print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)
        